# Trabalho Prático 1 de Estruturas Criptográficas

  - **Autores:** (Grupo 9)
      - Nelson Faria (A84727)
      - Miguel Oliveira (A83819)

## Exercício 2

In [36]:
# Imports necessários à execução deste notebook
import os, hashlib

### Alínea a - Construir uma classe Python que implemente um KEM-RSA

*A classe deve:*
  - *Inicializar cada instância recebendo  o parâmetro de segurança (tamanho em bits do módulo RSA) e gere as chaves pública e privada.*
  - *Conter funções para encapsulamento e revelação da chave gerada.*

Como o objetivo é o usao de um **KEM** baseado no **RSA**, aquilo que era necessário fazer eram funções de encapsulamento e desencapsulamento de uma chave simetrica a partir do uso das chaves públicas e privadas do **RSA**, que é um esquema de criptografia assimétrica. De seguida são esplicados todos os passos que efetuamos no código.

**Geração da chave pública $(n,e)$ e privada $(p,q,d)$ *(feito na inicialização da classe)***

Na função de inicialização da classe, aquilo que é feito é  o seguinte:

 - Geração de um número primo aleatório para o parâmetro $q$ do rsa no intervalo de $[2^{N-1};2^N-1]$;
 
 - Geração de um número primo aleatório para o parâmetro $p$ do rsa no intervalo de $[2^{N+1-1};2^{N+1}-1]$;
 
 - Calcular $n = p*q$;
 
 - Calcular $\phi(n) = (p-1) * (q-1)$;
 
 - Escolher um inteiro aleatorio $e$ entre 1 < e < $\phi(n)$, em que $e$ e $\phi(n)$ sejam relativamente primos;
 
 - Calcular o parâmetro $d$ com base no *Extended Euclidean algorithm* através da fórmula seguinte:
 
     $d.e−k.\phi(n)=1$
   
   Para descobrir então um $k$ e um $d$ que satisfaçam esta igualdade, aquilo que foi necessário fazer foi recorrer à identidade de *Bézout*: $ g=gcd(x,y)=sx+ty $. Como existe uma função *xgcd(x, y)* que retorna um triplo $(g, s, t)$, então $d$ será o resultado de:
    
     $d = mod(s,\phi(n))$, uma vez que $1 < d < \phi(n)$

**Encapsulamento e geração da chave *(função encapsula())***

Esta função recebe como input a chave pública $(e,n)$ e para a geração de uma chave simétrica e o seu respetivo 'encapsulamento', aquilo que foi necessário fazer foi:

 - Gerar um número inteiro $r$ aleatório entre 1 e $n-1$ (feito pela função *h(n)*);
 
 - Cálculo do 'encapsulamento' da chave através da expressão:
 
   $c ← r^e$ $mod$ $n$
 
 - Geração da chave simétrica a partir de $w ← KDF(r)$ (no nosso caso, usamos a biblioteca *hashlib* com um KDF que usa a função de hash SHA-256, daí ser necessário adicionalmente um *salt*)
 
 - Assim, como resultado teremos o par $(w,salt + c)$.

**Desencapsulamento da chave *(função desencapsula())***

A função *desencapsula()* recebe como argumento o 'encapsulamento' da chave $(salt + c)$ e realiza os seguintes passos:

 - Obter o valor $r$ através de:
 
   $r ← c^d$ $mod$ $n$
  
 - Com o $r$ pode-se então gerar a chave simétrica através de $w ← KDF(r)$, sendo este o output da função.

In [37]:
# Classe que implementa o KEM_RSA (Key Encapsulation Mechanism) baseado no RSA
# Baseamo-nos em: https://doc.sagemath.org/html/en/thematic_tutorials/numtheory_rsa.html
#                 https://docs.python.org/3/library/hashlib.html

class KEM_RSA(object):
    
    def __init__(self, N, timeout=None):
        
        # Obter um primo aleatorio para o parametro q do rsa no intervalo [2^(N-1);2^N-1]
        self.q = self.primoAleatorio(N)
        # Obter um primo aleatorio para o parametro p do rsa no intervalo [2^(N+1-1);2^(N+1)-1]
        self.p = self.primoAleatorio(N + 1)
        # n = p * q
        self.n = self.p * self.q
        # Funcao totiente de euler = (p-1)*(q-1)
        self.phi = (self.p-1)*(self.q-1)
        # Escolher um inteiro aleatorio e entre 1 < e < φ(n) e que e e φ(n) sejam relativamente primos
        self.e = ZZ.random_element(self.phi)
        while gcd(self.e, self.phi) != 1:
            self.e = ZZ.random_element(self.phi)
        
        # Para calcular d, usamos o 'extended Euclidean algorithm': de−k⋅φ(n)=1 -> Assim, so precisamos de descobrir d e -k
        # xgcd(x, y) retorna um triplo (g, s, t) que satisfaz a identidade de Bézout: g=gcd(x,y)=sx+ty 
        self.bezout = xgcd(self.e, self.phi)
        # d = mod(s,φ(n)), uma vez que 1 < d < φ(n)
        self.d = Integer(mod(self.bezout[1], self.phi))
        
          
    # Retorna um primo aleatorio entre 2^(N-1) e 2^N - 1
    def primoAleatorio(self, N):
        
        return random_prime(2**N-1,False,2**(N-1))
    
    
    # Funcao que gera um inteiro aleatorio entre 0 e n
    def h(self, n):
        
        # Escolher um inteiro aleatorio r entre 0 < r < n 
        r = ZZ.random_element(n)
        
        return r
    
    
    # Funcao que gera a chave e o seu 'encapsulamento' recebendo como parametro a seed do KDF, a chave publica (e,n) e
    # um salt usado no KDF
    def f(self, seed, e, n, salt):
        
        # Criptograma com este inteiro usado para o encapsulamento da chave (c ← seed^e mod n)
        c = Integer(power_mod(int(seed.decode()), e, n))
        
        # Geracao da chave simetrica a partir do seed (W ← KDF(r))
        w = hashlib.pbkdf2_hmac('sha256', seed, salt, 100000)
        
        return (w, salt + str(c).encode())

    
    # Funcao que serve para encapsular a chave que for acordada a partir de uma chave publica
    # Nota: Esta funcao foi decomposta em 2 por causa da alínea b)
    def encapsula(self, e, n):
        
        # Escolher um inteiro aleatorio r entre 0 < r < n 
        r = self.h(n)
        
        # Gerar o salt para derivar a chave
        salt = os.urandom(16)
        
        return self.f(str(r).encode(), e, n, salt)
    
    
    # Funcao usada para desencapsular uma chave, a partir do seu "encapsulamento"
    def desencapsula(self, cs):
        
        # Buscar os 16 primeiros bytes para obter o salt e o restante é o "encapsulamento" da chave
        salt = cs[:16]
        c = int(cs[16:].decode())
        
        # Obter o r (r ← c^d mod n) com o algoritmo power_mod
        r = Integer(power_mod(c, self.d, self.n))
        
        # Geracao da chave simetrica a partir do r (W ← KDF(r))
        w = hashlib.pbkdf2_hmac('sha256', str(r).encode(), salt, 100000)
        
        return w
        

**Testagem da classe definida acima:**

In [38]:
# Parametro de seguranca
N = 1024

# Chave publica: (n,e)
# Chave privada: (p,q,d)
# Inicializacao da classe responsavel por implementar o KEM-RSA
kemrsa = KEM_RSA(N)

# Verificar que ed == 1 (mod φ(n))
#print(mod(kemrsa.e * kemrsa.d, kemrsa.phi))

# Procede-se ao encapsulamento
(w,c) = kemrsa.encapsula(kemrsa.e, kemrsa.n)

print("Chave devolvida pelo encapsulamento: ")
print(w)
print("\n'Encapsulamento' da chave: ")
print(c)

# Procede-se ao desencapsulamento
w1 = kemrsa.desencapsula(c)
print("\nChave devolvida pelo desencapsulamento: ")
print(w1)
# Verificar se a chave devolvida pelo desencapsulamento e igual a que foi gerada no encapsulamento
if w == w1:
    print("As chaves são iguais!!!")
else:
    print("As chaves são diferentes!!!")

Chave devolvida pelo encapsulamento: 
b'\x94\x99\xd6\xc1\xa2\xfd\xa7\xeb^\x1f\xe0\xdb\x95\xed\x9f\xfbtf\xbff\xd2>\xc5_\xd5\x14N\xf0\xfe\xe5\xba\xf9'

'Encapsulamento' da chave: 
b'\xcf\x10\xb8\xe7\xe1\xf1\x8c\xce\x7f\xd8\xd2\xba\xb8\x90\x94\xc44147773440478560171301806924918526194161675842245513628597314387589901807541109464969503908693303338984686190277911693447491688793722706981499745367092062275179620836013085686979202374983424598999279801027674951662562732917266824560152787799445342582243784763058240486923536219260232856358395118973450431093103539395112430636556442992397294386196306292124602336461277212356124648215050019083370789692962934855140642889306418769048480221912494073226089847581620209232205991522152804641154234034891404868575195295536553830030970867731519443390602285800551432985259620970778955888320086670395461910031862068827611480866'

Chave devolvida pelo desencapsulamento: 
b'\x94\x99\xd6\xc1\xa2\xfd\xa7\xeb^\x1f\xe0\xdb\x95\xed\x9f\xfbtf\xbff\xd2>\xc5_\xd5\x14N\xf0\xf

### Alínea b - Construir,  a partir do KEM definido anteriormente e usando a transformação de Fujisaki-Okamoto, um PKE que seja IND-CCA seguro.

> Documentos em que nos baseamos:
>   - https://link.springer.com/content/pdf/10.1007/s00145-011-9114-1.pdf
>   - Apontamentos da aula que estão no *Dropbox Paper* (Capítulo 2)

Para construir um PKE (*Public Key Encryption*) que seja **IND-CCA** a partir do KEM que foi definido acima foi necessário alterar o código do **KEM-RSA** para que separa-se duas funções inportantes na fase de encapsulamento $(h,f)$ e ainda foi necessário seguir os seguintes passos:

**Geração das chaves pública e privada(feito na inicialização da classe)**

Para gerar a chave pública e privada para este esquema criptográfico, bastou-nos usar a geração de chave que acontece na inicialização do **KEM-RSA** definido acima. Assim, na inicialização da classe **PKE_IND_CCA** já é feito a inicialização da classe **KEM-RSA** e consequentemente já se obtêm as chaves pública e privada a partir daí.

**Cifragem(função *cifra()*)**

A função de cifragem recebe como input a mensagem a cifrar e a chave pública $(e,n)$ e efetua os seguintes passos:

- Primeiramente, este algoritmo seleciona um inteiro $r$ de forma aletória entre $[0,n]$;

- A seguir é feito o **XOR** entre a mensagem original e o hash do $r$ ($g(r)$), que deve ser do mesmo tamanho do que a mensagem original: y <- m $\oplus$ g(r);

- Depois é usada a função *f()* do **KEM-RSA** para encapsular uma chave que é gerada a partir do parâmetro $r$ e do $y$ e obtem-se o par $(k,w)$ que representam respetivamente a chave e o 'encapsulamento' da chave;

- Por fim ofusca-se a chave através do **XOR** entre o $r$ e o $k$: c <- r $\oplus$ k. Assim, o resultado final desta função é o triplo constituído por $(y,w,c)$.


**Decifragem (função *decifra()*)**

A função de decifragem recebe como input a ofuscação da mensagem original $y$, o encapsulamento da chave $w$ e a ofuscação da chave $c$ e realiza as operações seguintes:

 - Desencapsula a chave através do encapsulamento da chave $w$, sendo que esse desencapsulamento produz a chave $k$.
 
 - Calcula o resultado de r <- c $\oplus$ k
 
 - Teste se o resultado do encapsulamento da chave é igual a $(w,k)$, através dos parâmetros $r$ e $y$. Caso o resultado do encapsulamento da chave não for igual a $(w,k)$, então dá erro e lança exceção. Caso contrário é então calculado o resultado de m <- y $\oplus$ g(r), dando portanto a mensagem original como resultado.

In [39]:
# Classe que implementa um PKE_IND_CCA (Public Key Encryption) a partir do KEM_RSA e da transformação de Fujisaki-Okamoto
# Baseamo-nos em: https://doc.sagemath.org/html/en/thematic_tutorials/numtheory_rsa.html
#                 https://docs.python.org/3/library/hashlib.html

class PKE_IND_CCA(object):
    
    def __init__(self, N, timeout=None):
        
        # N é o tamanho usado para os primos p e q no RSA
        self.kem = KEM_RSA(N)
    
    
    # XOR de 2 arrays de bytes byte-a-byte! A mensagem(data) deve ser menor ou igual á chave(mask)! Caso contrario, a chave
    # ou mask é 'repetida' para os bytes seguintes dos dados
    def xor(self, data, mask):
        
        masked = b''
        ldata = len(data)
        lmask = len(mask)
        i = 0
        while i < ldata:
            for j in range(lmask):
                if i < ldata:
                    masked += (data[i] ^^ mask[j]).to_bytes(1, byteorder='big')
                    i += 1
                else:
                    break
                    
        return masked
    
    
    # Funcao usada para cifrar que recebe a mensagem e uma chave publica (e,n)
    def cifra(self, m, e, n):
        
        # Gerar um inteiro aleatorio r entre 0 < r < n
        r = self.kem.h(n)
        # Calculo do g(r), em que g é uma função de hash (no nosso caso, sha-256)
        g = hashlib.sha256(str(r).encode()).digest()
        # Efetuar o calculo de: y ← x⊕ g(r)
        y = self.xor(m, g)
        yi = Integer('0x' + hashlib.sha256(y).hexdigest())
        # Gerar o salt para derivar a chave
        salt = os.urandom(16)
        # Calcular (k,w) ← f(y || r)
        (k,w) = self.kem.f(str(yi + r).encode(), e, n, salt)
        # Calcular c ← k⊕ r
        c = self.xor(str(r).encode(), k)
        
        return (y,w,c)
    
    
    # Funcao usada para decifrar que recebe o criptograma, o 'encapsulamento' da chave, a tag e o vetor inicializacao
    def decifra(self, y, w, c):
        
        # Fazer o desencapsulamento da chave
        k = self.kem.desencapsula(w)
        # Calcula r <- c  ⊕  k
        r = self.xor(c, k)
        # Buscar os 16 primeiros bytes de w para obter o salt
        salt = w[:16]
        yi = Integer('0x' + hashlib.sha256(y).hexdigest())
        # Verificar se (w,k) ≠ f(y∥r)
        if (k,w) != self.kem.f(str(yi + int(r)).encode(), self.kem.e, self.kem.n, salt):
            # Lancar excecao
            raise IOError
        else:
            # Calculo do g(r), em que g é uma função de hash (no nosso caso, sha-256)
            g = hashlib.sha256(r).digest()
            # Calcular m <- y⊕ g(r)
            m = self.xor(y, g)
        
        return m
    

**Testagem da classe definida acima:**

In [40]:
pke = PKE_IND_CCA(1024)

#message = os.urandom(32)
message = b"TP1 de Estruturas Criptograficas"

print(b"Mensagem original: " + message)

# Cifragem da mensagem 
(y,w,c) = pke.cifra(message, pke.kem.e, pke.kem.n)

print("\nTexto cifrado:")
print(b"Y = " + y)
print(b"W = " + w)
print(b"C = " + c)

print("\nTexto decifrado:")

# Decifragem do criptograma
try:
    message1 = pke.decifra(y,w,c)
    print(b"Texto decifrado: " + message1)
    if message == message1:
        print("[A mensagem decifrada é igual à original!!!!]")
    else:
        print("[A mensagem decifrada é diferente da original!!!!]")
except IOError as e:
    print("Erro ao decifrar a mensagem!!!!")

b'Mensagem original: TP1 de Estruturas Criptograficas'

Texto cifrado:
b'Y = \xdd\xdc\xb2&\xb4}\xa7u\xf1\x89\x1f\xb2J\xae\x10\x1c\xd3\xa5VO\r\x82h\xc2\xd5\xd6\x04\xc9p\xc8hi'
b'W = \xd6\x06\xc1\x08\x94W\xcb\x9b\xd7\xb6I\xab\x17Q\xdb\x919466700548673341458077668928757876760796150366762214594455987855895311348806651935746167044442033015821488411899012713638635492879224252578676294597852885558127445414237808467175390707927328337004923370313801020512457946443352769174858348825704624613488088335715929464525644015435506816115214622930804799274367936937666685117740789400704168169331142980717053645103917868015327525223396623782856741287554313549382117239757009635565544937504499348143684065462666497075203479549145591088065543051959486071293560565440737097469782716219972771042170276801840322425282236550884377521463354680597081202443991039403552318350'
b"C = \x01\x87X\xd7a\x94\x18\x85\xb8O\xa95o&SJfx\x89{\x0b\x96\x8e\x8b\xe1\xcf4\x90\xa3\xf7\x9a\x07\x06\x8b\\\xdda\x90\x18\x80\xbcA\xae9k#RFc|\x89\

### Alínea c - Construir uma classe Python que implemente o DSA (*Digital Signature Algorithm*). 

*A implementação deve:*
  - *na inicialização,  receber como parâmetros o tamanho  dos primos $p$ e $q$;* 
  - *deve conter funções para assinar digitalmente e verificar a assinatura.*

Tendo em conta os requisitos descritos acima, procedemos então em seguida à descrição de todos os passos que efetuamos para a implementação do DSA.

**Criação do par de chaves pública e privada(feito na inicialização da classe)**

Para a criação do par de chaves pública e privada $(y,x)$, é necessário primeiro criar os parâmetros $(p,q,g)$ e só depois é que se pode efetuar o cálculo das chaves pública e privada. Deste modo, a função de inicialização da classe recebe como parâmetros $L$ e $N$ que são respetivamente o tamanho em bits dos parâmetros $p$ e $q$ e depois procede à criação de todos os valores referidos anteriormente, de acordo com o que será apresentado de seguida.

- **Geração dos parâmetros $(p,q,g)$**

   - Primeiro foi necessário gerar o número primo $q$ de forma aleatória entre $[2^{N-1};2^N]$;
   - De seguida, procedeu-se ao cálculo do número primo $p$ também de forma aleatória, porém o senão é que para que o algoritmo funcione corretamente, $p-1$ têm de ser múltiplo de $q$. Existem várias formas de fazer isto, porém a forma que escolhemos será descrita de seguida:
   
   Se dissermos que $q$ é então um número primo aleatório, então p = $2*q + 1$ pode ser testado para ver se é primo, sendo que tem ainda garantida a característica de $p-1$ ser múltiplo de $q$. Dentro disto, aquilo que fizemos foi precisamente isto, mas de forma a que este valor esteja no intervalo $[2^{L-1},2^{L}]$ e, além disso seja primo e aleatório!!!
   
   Por isso, o que se pensou fazer para $p$ foi múltiplicar o 2 por um número aleatório entre $[1,2^{L-N-2}]$ e somar-lhe o limite inferior para o parâmetro $p$, pelo que depois foi só seguir a seguinte regra da matemática:
   $2^X + 2*(2^Y) <=> 2^{X+Y+1}$
   
   Se X = 47 e Y = 16, então $2^{47} + 2*(2^{16}) = 2^{47+16+1} = 2^{64}$
   
   Com isto conseguimos arranjar a seguinte solução mais ou menos eficiente: 
   
   $p := (2^{L-N-1} + 2*random(1,2^{L-N-2})) * q + 1$ 
   
   Assim, conseguimos ter um número aleatório e primo no intervalo de $[2^{L-1},2^{L}]$, visto que q têm tamanho $[2^{N-1};2^N]$;
   
   - Por fim, só falta calcular o gerador $g$, pelo que para isso foi só necessário ter em conta que este não pode ser igual a 1 ($g != 1$) e ainda a seguinte fórmula: 
   
   $g := h^{(p-1) / q}$
   
   Em que $h$ é um número inteiro aleatório entre $[1;p-1]$.

- **Geração do par de chaves pública e privada $(y,x)$**

  - Para gerar a chave privada $x$ basta gerar um número inteiro aleatório entre $[1,q-1]$;
  
  - A chave pública $y$ é o resultado da seguinte expressão:
  
   $y := g^x$ $mod$ $p$

**Assinatura digital da mensagem(função *assina()*)**

Depois de criados todos os parâmetros na inicialização da classe, bem como as chave pública e privada, já se pode assinar qualquer mensagem da escolha do utilizador. O resultado da função *assina()* é o par $(r,s)$ que representa a assinatura e é calculado da seguinte forma:

- Geração de um inteiro $k$ aleatoriamente entre $[1,q-1]$;

- Calcular o valor $r$ com base na seguinte fórmula:

 $r := (g^k$ $mod$ $p)$ $mod$ $q$, em que $r != 0$

- Calcular o valor $s$ com base na seguinte fórmula:
 
 $s := (k^{-1} (H(m) + xr))$ $mod$ $q$, em que $s != 0$

**Verificação da assinatura digital(função *verifica()*)**

Por fim, a função $verifica()$ serve para verificar se uma determinada assinatura é válida. Por isso, esta recebe como parâmetros a assinatura, a mensagem original e ainda a chave pública $y$ da entidade que assinou a mensagem. Assim, os passos que são efetuados por esta função são os seguintes:

- Primeiro a função verifica se os parâmetros $r$ e $s$ da assinatura estão entre $[0,q]$;
- Calcular $w$ que é o resultado de:
 
 $w := s^{-1} mod$ $q$
 
- Calcular $u1$ que é o resultado de:

 $u1 := H(m).w$ $mod$ $q$

- Calcular $u2$ que é o resultado de:

 $u2 := r.w$ $mod$ $q$

- Calcular $v$ que é o resultado de:

 $v := (g^{u1} . y^{u2} mod$ $p)$ $mod$ $q$

- Por fim, se $v == r$, então a assinatura é válida!

In [41]:
# Classe que implementa o algoritmo de assinatura digital(DSA) 
# Baseamo-nos em: https://en.wikipedia.org/wiki/Digital_Signature_Algorithm
#                 https://doc.sagemath.org/html/en/developer/coding_in_python.html

class DSA(object):
    
    def __init__(self, L, N, timeout=None):
        
        # Escolha de um numero primo q grande
        self.q = Integer(self.primoAleatorio(N))
        # Escolha de um numero primo p grande, em que p-1 seja multiplo de q
        self.p = 1
        while (not is_prime(self.p)):
            # Como 2^(L-1) < p < 2^L e 2^(N-1) < q < 2^N, seguimos a logica seguinte: 2^(L-N) * 2^N = 2^(L-N+N) = 2^L
            # Além disso, Se n for aleatorio, 2*n + 1 tambem é
            # p := (2^(L-N-1) + 2*random(1,2^(L-N-2))) * q + 1
            self.p = (2^(L-N-1) + 2*ZZ.random_element(1, 2^(L-N-2)))*self.q + 1  
        # calcular o gerador g tal que tal que g := h^(p-1)/q mod p, com , com g != 1
        self.g = 1
        while self.g == 1:
            # Escolher um inteiro aleatorio h entre 1 < h < p-1 
            h = ZZ.random_element(2, self.p-1)
            self.g = power_mod(h, (self.p-1) // self.q, self.p)
        # Gerar a chave privada x aleatoriamente entre 1 e q-1
        self.x = ZZ.random_element(1, self.q)
        # Gerar a chave pública y := g^x mod p
        self.y = Integer(power_mod(self.g, self.x, self.p))
    
    
    # Retorna um primo aleatorio entre 2^(N-1) e 2^N
    def primoAleatorio(self, N):
        
        return random_prime(2**N,False,2**(N-1))
    
    
    # Funcao usada para assinar uma determinada mensagem
    def assina(self, message):
        
        # Criacao do par da assinatura DSA (r,s)
        r = 0   # r != 0
        s = 0   # s != 0
        while r == 0 or s == 0:
            # Gerar um inteiro aleatorio k entre 1 e q-1
            k = ZZ.random_element(1,self.q)
            # r := (g^k mod p) mod q, com r != 0
            r = Integer(mod(power_mod(self.g, k, self.p), self.q))
            # s := (k^-1 (H(m) + xr)) mod q, com s != 0
            s = Integer(mod(inverse_mod(k, self.q) * mod((hash(message) + self.x*r), self.q), self.q))
    
        # Assinatura = (r,s)
        return (r,s)
    
    
    # Funcao que verifica uma determinada assinatura a partir da mensagem original e da chave publica 
    def verifica(self, signature, message, y):
        
        # Retirar os valores de r e s
        r = signature[0]
        s = signature[1]
        # Verificar que os valores de r e s estao entre 0 e q
        if r > 0 and r < self.q and s > 0 and s < self.q:
            
            # Calcular w := s^-1 mod q
            w = Integer(inverse_mod(s, self.q))
            # Calcular u1 := H(m).w mod q
            u1 = Integer(mod(hash(message) * w, self.q))
            # Calcular u2 := r.w mod q
            u2 = Integer(mod(r * w, self.q))
            # Calcular v := (g^u1 * y^u2 mod p) mod q
            v = Integer(mod(mod(power_mod(self.g, u1, self.p) * power_mod(y, u2, self.p), self.p), self.q))
            print("V = " + str(v))
        
        else:
            return False
        
        return v == r
        

**Testagem da classe definida acima:**

In [42]:
# Tamanhos para os parametros p e q
L = 2048
N = 256

dsa = DSA(L,N)

print("P = " + str(dsa.p))
print("Q = " + str(dsa.q))

# Teste de uma assinatura e verificacao
message = input("\nInsira a mensagem a assinar:")
assinatura = dsa.assina(message.encode("utf-8"))
print("\nR = " + str(assinatura[0]))
print("S = " + str(assinatura[1]))
if dsa.verifica(assinatura, message.encode("utf-8"), dsa.y):
    print("A assinatura é válida!!!!")
else:
    print("A assinatura é inválida!!!!")


P = 9759384961370659717889070666527502895936215130102265506664095998852961364207084683218110221293479921460458291292795753717352137366080254826858597768427965909787793839278539646830779951504474697231914524263074060082859194887836946073518913796205132965815485370274307058672037691563408249931346943189795676064761433419117449909378615846745200029809767581421266970200917562802875818347208749390516714674315678157727514547139021003977451784047555019052678327547378288629256368682751458820282763182482199013048297219237294682785583336604765649054872210136393972374005707712362473128282806128542462163589016966493795893263
Q = 58460040422232779322391683905138235342727141679860297925885151970338870275003

Insira a mensagem a assinar:TP1 de Estruturas Criptograficas

R = 43966899693750245568729444114009216638685190968415137119852571844454081237677
S = 17267298084724179730991605721713726063721873929971340102566300687642735723289
V = 439668996937502455687294441140092166386851909684151371198525718444

### Alínea d -  Construir uma classe Python que implemente o ECDSA (*Elliptic Curve Digital Signature Algorithm*) usando uma das curvas elípticas primas definidas no FIPS186-4  (escolhida  na iniciação da classe).

Documento em que nos baseamos: https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.186-4.pdf

Mais referencias: https://pt.wikipedia.org/wiki/ECDSA

De acordo com o ficheiro mencionado acima da *FEDERAL INFORMATION PROCESSING STANDARDS PUBLICATION* (**FIPS PUB 186-4**) sobre standards de assinaturas digitais, existem várias opções para a escolha da curva elítica prima. A curva elítica que escolhemos foi a **P-256**, sendo que esta possuí os seus próprios valores standard para os parâmetros da curva descritos nesse mesmo documento. Estes parâmetros assumem um papel fundamental e representam, de forma sucinta, a curva, o ponto base da curva $G$ e ainda $n$, que é a ordem inteira de $G$ (notar que $n*G = O$, sendo $O$ o elemento identidade). Assim, procedemos aos vários passos do DSA, mas em modo de curvas elíticas.

**Criação do par de chaves pública e privada(feito na inicialização da classe)** (baseado na pág 62)

A chave pública $Q$, a chave privada $d$  e os parâmetros do domínio da curva elítica estão matemáticamente relacionados uns com os outros. Deste modo, para a criação deste par de chaves é necessário:

  - Chave Privada $d$: é um inteiro gerado de forma aleatória no intervalo de [1,n-1]
  - Chave Pública $Q$: dado um ponto gerador (ou ponto base) $G$ da curva elítica, $Q$ é um ponto na curva elítica que obedece à seguinte relação: $Q := d * G$
  
**Assinatura digital da mensagem(função *assina()*)** (baseado na página da wikipedia)

De seguida serão descritos todos os passos que foram efetuados para assinar $(r,s)$ um dada mensagem recebida como input.

   - Calcular $e = HASH(m)$ (no nosso caso a função de hash que vamos usar é o SHA-256)
   - Considerar $z$ como sendo os $L_{n}$ bits mais à esquerda de $e$, em que $L_{n}$ é o comprimento de bit da ordem do grupo $n$. (Nota só para o facto de que $z$ pode ser maior do que $n$ mas não mais longo. Além disso, referir o facto de como o hash que usamos é o SHA-256 e a curva possuí um $n$ de 256 bits, então no nosso caso $z==e$)
   - Geração de um número inteiro aleatório $k$ entre o intervalo [1,n-1];
   - Calcular um ponto na curva tal que: $(x1, y1) = k * G$, em que $G$ continua a ser um ponto gerador da curva que foi definido inicialmente;
   - Calcular então o valor de $r$ tal que $r = x1$ $mod$ $n$. Se $r = 0$, então voltar a gerar um $k$ e consequentemente o ponto $(x1,y1)$;
   - Por fim, calcular o valor de $s$ que é igual a $s = k^{-1} * (z + r*d)$ $mod$ $n$. Se $s = 0$, então voltar a gerar um $k$ e consequentemente o ponto $(x1,y1)$;
   - A assinatura é o par $(r,s)$. $(E (r,-s$ $mod$ $n)$ também é uma assinatura válida.)

Um dos pontos importantes é a geração do número aleatório $k$. Se duas mensagens diferentes utilizarem na geração um $k$ igual, podem ocorrer ataques que podem resultar na extração da chave de assinatura. Assim, para garantir que $k$ é único para cada mensagem, pode-se ignorar completamente a geração de números aleatórios e gerar assinaturas determinísticas derivando $k$ tanto da mensagem quanto da chave privada.

**Verificação da assinatura digital(função *verifica()*)** (baseado na página da wikipedia)

Antes de mais nada, para efetuar a verificação da assinatura de uma dada mensagem, deve-se ter uma cópia da chave pública de ponto da curva $Q$. Nós podemos verificar que $Q$ é um ponto válido da curva da seguinte forma:

   - Verificar que $Q$ não é igual ao elemento identidade $O$, e as suas coordenadas são, de outra forma válida;
   - Verificar que $Q$ está sobre a curva;
   - Verificar que $n * Q = O$.

Depois de verificarmos isto, podemos seguir para os passos seguintes de verificação da assinatura:

   - Verificar que $r$ e $s$ são números inteiros em $[1,n-1]$. Se não, a assinatura é inválida;
   - Calcular $e = HASH(m)$ (este hash que é feito aqui deve ser o mesmo que foi usado na assinatura da mensagem);
   - Considerar $z$ como os $L_{n}$ bits mais à esquerda de $e$;
   - Calcular $w=s^{-1} $$mod$ $n$;
   - Calcular $u_{1}=zw$ $mod$ $n$;
   - Calcular $u_{2}=rw$ $mod$ $n$;
   - Calcular o ponto da curva $(x_{1},y_{1})=u_{1} * G+u_{2} * Q$. Se $(x_{1},y_{1})=O$, então a assinatura é inválida.
   - A assinatura é válida se $r\equiv x_{1}$ $mod$ $n$, e inválida caso contrário.

In [43]:
# Classe que implementa o Algoritmo de Assinatura Digital de Curvas Elípticas(ECDSA)
# Baseado no documento(durante o codigo, tem referencias a paginas deste documento): 
# https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.186-4.pdf
# https://doc.sagemath.org/html/en/reference/finite_rings/sage/rings/finite_rings/finite_field_constructor.html
# https://doc.sagemath.org/html/en/reference/arithmetic_curves/sage/schemes/elliptic_curves/constructor.html

class ECDSA(object):
    
    def __init__(self, timeout=None):
        
        # Parâmetros da Curva Elíptica P-256 (ver pag. 91)
        # p e n são números primos de 256 bits
        self.p = 115792089210356248762697446949407573530086143415290314195533631308867097853951
        self.n = 115792089210356248762697446949407573529996955224135760342422259061068512044369
        # A seleção a ≡ -3 para o coeficiente de x foi feita por razões de eficiência(ver pag.89); consulte IEEE Std 1363-2000
        self.a = -3
        # Este parametro esta em hexadecimal
        self.b = 0x5ac635d8aa3a93e7b3ebbd55769886bc651d06b0cc53b0f63bce3c3e27d2604b
        # Este parametro esta em hexadecimal
        self.Gx = 0x6b17d1f2e12c4247f8bce6e563a440f277037d812deb33a0f4a13945d898c296
        # Este parametro esta em hexadecimal
        self.Gy = 0x4fe342e2fe1a7f9b8ee7eb4a7c0f9e162bce33576b315ececbb6406837bf51f5
        
        # Criacao do corpo finito em torno do numero primo p
        Fp = GF(self.p)
        # Equacao de Weierstrass:  y^2 + a_1xy + a3y = x^3 + a2^2 + a4x + a6
        # Curva Elíptica E sobre o corpo finito Fp, onde a1=a2=a3=0, a4 = a mod p e a6 = b mod p
        self.E = EllipticCurve(Fp, [self.a, int(self.b)])
        # Ponto Gerador G pertencente à Curva Elíptica E
        self.G = self.E((self.Gx,self.Gy))
        
        # Criacao do par de chaves publica e privada (Q,d)
        # Chave privada -> inteiro gerado de forma aleatória no intervalo de [1,n-1]
        self.d = ZZ.random_element(1, self.n)
        # Chave publica -> Q := d * G
        self.Q = self.d * self.G
        
    
    # Funcao usada para assinar uma determinada mensagem
    def assina(self, message):
        
        # Calcular e=HASH(m) 
        e = Integer('0x' + hashlib.sha256(message).hexdigest())
        # Criacao do corpo finito em torno do numero primo n
        Fn = GF(self.n)
        # r e s nao podem ser 0, sendo unicamento inicializado a 0 para controlar o ciclo
        r = 0
        s = 0
        while r==0 or s==0:
            
            # Geracao de um numero aleatorio k no intervalo de [1,n-1]
            k = ZZ.random_element(1, self.n)
            # (x1,y1)=k∗G
            (x1,y1) = (k * self.G).xy()
            # Calcular r tal que r=x1  mod n, em que r!=0 (Decimos aplicar um corpo finito em torno de n, que vai ter ao mesmo)
            r = Integer(Fn(x1))
            # Calcular s tal que s=k−1∗(z+r∗d) mod n, em que s!=0. No nosso caso, z==e
            s = Fn(k^-1 * (e + r*self.d))
        
        # Assinatura = (r,s)
        return (r,s)
    
    
    # Funcao que verifica uma determinada assinatura a partir da mensagem original e da chave publica
    def verifica(self, signature, message, Q):
        
        # Antes de verificar a assinatura, verificar o Q recebido como parâmetro (sabendo que n*G = O)
        if Q == self.n*self.G or self.n*Q != self.n*self.G:
            return False
        # Verificar de Q pertence à curva
        try:
            self.E(Q)
        except TypeError as e:
            return False
        # Retirar os valores de r e s
        r = signature[0]
        s = signature[1]
        # Verificar que r e s são números inteiros em [1,n−1]. Se não, a assinatura é inválida
        if r>=1 and r<=self.n-1 and s>=1 and s<=self.n-1:
            
            # Calcular e=HASH(m), que deve ser feito da mesma forma que foi feito na assinatura
            e = Integer('0x' + hashlib.sha256(message).hexdigest())
            # Criacao do corpo finito em torno do numero primo n
            Fn = GF(self.n)
            # Calcular w=s−1 mod n
            w = Fn(s^-1)
            # Calcular u1=zw mod n, no nosso caso, z==e
            u1 = Integer(Fn(e*w))
            # Calcular u2=rw mod n
            u2 = Integer(Fn(r*w))
            # Calcular (x1,y1)=u1∗G+u2∗Q
            XY = u1*self.G + u2*self.Q
            (x1,y2) = XY.xy()
            if XY == self.n*self.G:
                return False
            
        else:
            return False
        
        print("Fn(x1): " + str(Fn(x1)))
        
        return r == Fn(x1)
        

**Testagem da classe definida acima:**

In [44]:
# Iniciar a classe
ecdsa = ECDSA()

print("Chave Pública Q = " + str(ecdsa.Q))
print("Chave Privada d = " + str(ecdsa.d))

# Teste de uma assinatura e verificacao
message = input("\nInsira a mensagem a assinar:")
assinatura = ecdsa.assina(message.encode("utf-8"))
print("\nR = " + str(assinatura[0]))
print("S = " + str(assinatura[1]))
if ecdsa.verifica(assinatura, message.encode("utf-8"), ecdsa.Q):
    print("A assinatura é válida!!!!")
else:
    print("A assinatura é inválida!!!!")

Chave Pública Q = (108025465549353099414079940397151454931155274872251859332908526558702337914495 : 56218282041118118700588658253669885367874175740579059629135144117511387690677 : 1)
Chave Privada d = 72903442061196107195631345551891967068423227485366266634880544661778093512853

Insira a mensagem a assinar:TP1 de Estruturas Criptograficas

R = 77193115389203752729936916117949791414529177945377121755274456214258530844858
S = 24639838334910130389942642553618595972193937672934957398022901916402591567767
Fn(x1): 77193115389203752729936916117949791414529177945377121755274456214258530844858
A assinatura é válida!!!!
